### 📝 Client Request: Option Purchase

A client has requested that the bank **purchase an American put option** on **Commonwealth Bank of Australia (CBA)** with the following details:

- **Option Type**: American Put  
- **Underlying Asset**: Commonwealth Bank of Australia (CBA)  
- **Strike Price**: $170.00  
- **Expiry Date**: 15 May 2026